## Introduction


Segmentation — identifying which image pixels belong to an object — is a core task in computer vision and is used in a broad array of applications, from analyzing scientific imagery to editing photos. But creating an accurate segmentation model for specific tasks typically requires highly specialized work by technical experts with access to AI training infrastructure and large volumes of carefully annotated in-domain data.

source - https://ai.facebook.com/blog/segment-anything-foundation-model-image-segmentation/

First make sure pytorch and torchcvision are installed, for GPU
In my case: 
- pip install torch torchvision torchaudio
- pip install opencv-python matplotlib
- pip install 'git+https://github.com/facebookresearch/segment-anything.git'
    - OR download the repo locally and install
    - and:  pip install -e .

- Download the default trained model: 
    - https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
- Other models are available:
    - https://github.com/facebookresearch/segment-anything#model-checkpoints


In [1]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

PyTorch version: 2.0.0
Torchvision version: 0.15.1
CUDA is available: False


In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2


In [3]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor